In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

import numpy as np
from datetime import datetime
from packaging import version
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard

from tensorflow_model.efficientnet import EfficientNet

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.3.0 or above."

TensorFlow version:  2.3.0


# EfficientNet-B2

In [2]:
# Change to other families by referencing: 
# ~\miniconda3\envs\carnd\Lib\site-packages\tensorflow\python\keras\applications\efficientnet.py

def EfficientNetB2(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   classifier_activation='softmax',
                   **kwargs):
    return EfficientNet(
        1.1,
        1.2,
        260,
        0.3,
        model_name='efficientnetb2',
        include_top=include_top,
        weights=weights,
        input_tensor=input_tensor,
        input_shape=input_shape,
        pooling=pooling,
        classes=classes,
        classifier_activation=classifier_activation,
        **kwargs)

In [3]:
img_input_data = keras.layers.Input(shape=(12, 128, 256), name="input_imgs")
permute1 = keras.layers.Permute((2,3,1))(img_input_data)
efficientnet = EfficientNetB2(include_top=False, weights = None, input_shape=(128,256,12))(permute1)
conv2d1 = tf.keras.layers.Conv2D(32, 3, padding='same')(efficientnet)
normalization = keras.layers.BatchNormalization()(conv2d1)
elu1 = tf.keras.layers.ELU(alpha=1.0)(normalization)
flatten1 = tf.keras.layers.Flatten(name="vision_features")(elu1)


dense1 = tf.keras.layers.Dense(512)(flatten1)
dense1_relu = tf.keras.layers.ReLU(name="dense1_relu")(dense1)
dense2 = tf.keras.layers.Dense(256)(dense1_relu)
dense2_relu = tf.keras.layers.ReLU(name="dense2_relu")(dense2)
dense3 = tf.keras.layers.Dense(128)(dense2_relu)
dense3_relu = tf.keras.layers.ReLU(name="dense3_relu")(dense3)
path = tf.keras.layers.Dense(200)(dense3_relu)

output = path
model = keras.models.Model(inputs=[img_input_data], outputs=[output])

# Renaming all layers which deals with image
"""
for layer in model.layers:
    layer._name = "vision_" + layer.name
"""
    
#concat = keras.layers.concatenate([input_data, hidden2])


print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_imgs (InputLayer)      [(None, 12, 128, 256)]    0         
_________________________________________________________________
permute (Permute)            (None, 128, 256, 12)      0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 4, 8, 1408)        7771179   
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 8, 32)          405536    
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 8, 32)          128       
_________________________________________________________________
elu (ELU)                    (None, 4, 8, 32)          0         
_________________________________________________________________
vision_features (Flatten)    (None, 1024)             